# Deteccion de movimiento

importar librerias

In [2]:

import numpy as np
import cv2


secuencia

In [5]:
frame.shape

(480, 640, 3)

In [22]:
## llamamos al video
cap=cv2.VideoCapture(0)

#usamos algoritmo de sustraccion de fondo
fgbg=cv2.bgsegm.createBackgroundSubtractorMOG()

""""se aplica el kernel manualmente con esta funcion a la cual el kernel
queda asi 

 cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)) depende del tamaño
array([[0, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0]], dtype=uint8)
"""""
kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))


#iniciamos loop

while True:
    ##lee la entrada del video en si como booleano y "frame" es la imagen a sacar con frames
    ret,frame =cap.read()
    ##si no  es verdadero ret se quita
    if ret==False:
        break
    
    #dejamos la entrada de imagen en grises
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #redimensionar la imagen si es necesario
    #cv2.resize(frame,(10,10),interpolation=cv2.INTER_AREA)
    
    """""
    Se inician rectangulos en donde sera puesta la imagen, donde empieza en (x,y),donde termina(x,y)
    frame.shape[1] es el valor de 1920 en x, luego colores en rgb de 0 a 255, y el contorno que tendra

    """""
    cv2.rectangle(frame,(0,0),(frame.shape[1],40),(0,0,0),-1)
    color=(0,255,0)

    # se pone un mensaje en el rectangulo
    texto_estado="Estado: No se ha detectado movimiento"

    """""
    Definimos el area a usar [x,y] del primer punto de izquierda superior,derecha superior,derecha inferior,
    izquierda inferior

    esta configuracion es el area que vamos a detectar dentro de la imagen

    se utiliza numpy para crear dicha matriz

    """""
    #area_pts=np.array([[650,700],[1200,700],[1500,frame.shape[0]],[500,frame.shape[0]]])
    area_pts=np.array([[100,100],[600,100],[600,frame.shape[0]],[100,frame.shape[0]]])

    #ponemos en ceros una matriz de 1440 filas,1920 columnas
    imAux=np.zeros(shape=(frame.shape[:2]),dtype=np.uint8)
    #dentro de la matriz auxiliar dibujamos un controrno en si(en donde se dibuja, en que puntos seran)
    imAux=cv2.drawContours(imAux,[area_pts],-1,(255,255,255),-1)
    #convertimos el area blanca en gris para tener l a imagen con contrsates
    image_area=cv2.bitwise_and(gray,gray,mask=imAux)

    ## creamos otra imagen llamada fgmask el cual se sustrae de
    fgmask=fgbg.apply(image_area)

    # se crean transformaiones de la segmentacion en el cual quitamos ruido
    fgmask=cv2.morphologyEx(fgmask, cv2.MORPH_OPEN,kernel)
 
    ##rellenamos las segmentaciones
    fgmask=cv2.dilate(fgmask,None,iterations=5)

    #relleno de segmentacion de puntos negros
    fgmask =cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)
    
    """""
    Basandonos de fgmask buscamos si hay contornos es decir las segmentaciones o si se ve eso 

    """"" 
    cnts=cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]

    """""
    se hace ciclo for para analizar dentro de  elldibujo de contornos y si es mayor a 500 piixeles es decir su area
    se dibujara el  cuadro dentro del area en movimiento y dara mensaje de que hay movimiento y cambia el cuadro
    a color rojo  y cambia el estado y su solor 

    """""
    for cnt in cnts:
        if cv2.contourArea(cnt)>100:
            #retorna  los valores que vwemos, 
            x,y,w,h=cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            texto_estado="Estado alerta movimiento detectado!"
            color=(0,0,255)

    """""
    dibujamos en contrno en los puntos que hemos asignado sin rellenar con el color puesto, agregamos texto en el 
    nuevo frame, y mostramos al final las dos salidas, una del video y otra segmentados

    """""
    cv2.drawContours(frame,[area_pts],-1,color,2)
    cv2.putText(frame, texto_estado,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,color,2)
    cv2.imshow('fgmask',fgmask)
    cv2.imshow("frame",frame)

    # si queremos salir necesitamos apretar ESC
    k=cv2.waitKey(70) & 0xFF
    if k==27:
        break

#cierre de video y ventana
cap.release()
cv2.destroyAllWindows()
